In [1]:
import cv2 as cv
import os
import numpy as np

## Operatii cu video-uri

In [2]:
def display_video(video_path, window_name='Video', frame_delay=25):
    cap = cv.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break
        cv.imshow(window_name, frame)
        if cv.waitKey(frame_delay) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()
    
video_path = "video.mp4"
display_video(video_path)

In [3]:
def extract_frames(video_path, output_folder):
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break
        
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()

video_path = "video.mp4"
output_folder = "frames"
extract_frames(video_path, output_folder)

End of video.


### Extragerea foreground-ului folosind diferenta dintre frame-uri

In [4]:
def foreground_extraction_frame_difference(image_folder, output_video, output_video2, fps=25):

    images = sorted([img for img in os.listdir(image_folder) if img.endswith(('.jpg'))])

    first_frame = cv.imread(os.path.join(image_folder, images[0]))

    height, width, _ = first_frame.shape
    video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    previous_frame = None
    for image_name in images:
        current_frame = cv.imread(os.path.join(image_folder, image_name))
        if previous_frame is not None:
            frame_diff = cv.absdiff(current_frame, previous_frame)
            gray_frame = cv.cvtColor(frame_diff, cv.COLOR_BGR2GRAY)
            _, binary_diff = cv.threshold(gray_frame, 30, 255, cv.THRESH_BINARY)
            binary_diff_bgr = cv.cvtColor(binary_diff, cv.COLOR_GRAY2BGR)
            video.write(binary_diff_bgr)
            color_image = cv.bitwise_and(current_frame, binary_diff_bgr)
            video2.write(color_image)
        previous_frame = current_frame

    video.release()
    video2.release()

image_folder = 'frames'
output_video = 'frame_diff.mp4'
output_video2 = 'frame_diff_color.mp4'
foreground_extraction_frame_difference(image_folder, output_video, output_video2, fps=25)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:661: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


### Extragerea unei imagini de background

In [11]:
def background_extraction(image_folder, method, k=None, x=None, y=None):
    
    images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])
    
    selected_frames = []
    if method == 'mean_first_k':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(k)]
    
    elif method == 'median_first_k':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(k)]
    
    elif method == 'mean_range':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(x, y + 1)]
    
    elif method == 'median_range':
        selected_frames = [cv.imread(os.path.join(image_folder, images[i])) for i in range(x, y + 1)]
    
    else:
        raise ValueError("Invalid method")
    
    if method.startswith('mean'):
        background = np.mean(selected_frames, axis=0).astype(np.uint8)
    elif method.startswith('median'):
        background = np.median(selected_frames, axis=0).astype(np.uint8)
    
    return background

### Extragerea foreground-ului folosind background fix

In [17]:
def foreground_extraction_fixed_background(image_folder, method, output_video, output_video2, k=30, x=None, y=None, fps=25, processing=False):
	
	images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])

	first_frame = cv.imread(os.path.join(image_folder, images[0]))
	height, width, _ = first_frame.shape

	video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
	video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

	background = background_extraction(image_folder, method, k, x, y)
	for image_name in images:
		current_frame = cv.imread(os.path.join(image_folder, image_name))
		frame_diff = cv.absdiff(current_frame, background)
		gray_frame = cv.cvtColor(frame_diff, cv.COLOR_BGR2GRAY)
		_, binary_diff = cv.threshold(gray_frame, 40, 255, cv.THRESH_BINARY)
		binary_diff_bgr = cv.cvtColor(binary_diff, cv.COLOR_GRAY2BGR)
		video.write(binary_diff_bgr)
		color_image = cv.bitwise_and(current_frame, binary_diff_bgr)
		video2.write(color_image)

	video.release()
	video2.release()

image_folder = 'frames'
output_video = 'mean_k.mp4'
output_video2 = 'mean_k.mp4'
foreground_extraction_fixed_background(image_folder, 'mean_first_k', output_video, output_video2, k=30, x=470, y=480, fps=25)

### Extragerea foreground-ului folosind background adaptiv

In [18]:
def foreground_extraction_adaptive_background(image_folder, method, output_video, output_video2, k=10,fps= 25,processing=False):
	
	images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])

	first_frame = cv.imread(os.path.join(image_folder, images[0]))
	height, width, _ = first_frame.shape

	video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
	video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

	count = k
	for image_name in images[k:]:
		background = background_extraction(image_folder, method, k, x=count-k, y=count-1)
		current_frame = cv.imread(os.path.join(image_folder, image_name))
		frame_diff = cv.absdiff(current_frame, background)
		gray_frame = cv.cvtColor(frame_diff, cv.COLOR_BGR2GRAY)
		_, binary_diff = cv.threshold(gray_frame, 40, 255, cv.THRESH_BINARY)
		binary_diff_bgr = cv.cvtColor(binary_diff, cv.COLOR_GRAY2BGR)
		count += 1
		video.write(binary_diff_bgr)
		color_image = cv.bitwise_and(current_frame, binary_diff_bgr)
		video2.write(color_image)
		
	video.release()
	video2.release()

image_folder = 'frames'
output_video = 'adaptive.mp4'
output_video2 = 'adaptive_color.mp4'
foreground_extraction_adaptive_background(image_folder, 'median_range', output_video, output_video2, k=10, fps=25)

### Numararea obiectelor din video

In [19]:
def calculate_iou(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)

    if x_right <= x_left or y_bottom <= y_top:
        return 0  

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - intersection_area

    iou = intersection_area / union_area
    return iou

In [25]:
image_folder = 'frames'
output_video = 'count_cars.mp4'
output_video2 = 'count_cars_color.mp4'
fps = 25

images = sorted([img for img in os.listdir(image_folder) if img.endswith('.jpg')])

first_frame = cv.imread(os.path.join(image_folder, images[0]))
height, width, _ = first_frame.shape

video = cv.VideoWriter(output_video, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
video2 = cv.VideoWriter(output_video2, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

background = background_extraction(image_folder, 'median_range', k=30, x=470, y=480)

previous_contours = []  
total_passed_count = 0  

for image_name in images:
	current_frame = cv.imread(os.path.join(image_folder, image_name))
	frame_diff = cv.absdiff(current_frame, background)
	gray_frame = cv.cvtColor(frame_diff, cv.COLOR_BGR2GRAY)
	_, binary_diff = cv.threshold(gray_frame, 40, 255, cv.THRESH_BINARY)
	binary_diff = cv.dilate(binary_diff, (5, 5))
	binary_diff_bgr = cv.cvtColor(binary_diff, cv.COLOR_GRAY2BGR)
	middle = height // 2
	cv.line(current_frame, (0, middle), (width, middle), (0, 0, 255), 2)
	contours, _ = cv.findContours(binary_diff, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
	new_contours = []

	for contour in contours:
		x, y, w, h = cv.boundingRect(contour)
		if cv.contourArea(contour) >= 700:
			center_bbox = y + h // 2
			if middle - 10 <= center_bbox <= middle + 10:
				new = True
				for previous_contour in previous_contours:
					iou = calculate_iou(previous_contour, (x, y, w, h))
					if iou > 0.5:
						new = False
						break
				if new:
					total_passed_count += 1
				new_contours.append((x, y, w, h))
				cv.rectangle(current_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

	previous_contours = new_contours.copy()
		
	cv.putText(current_frame, f'{total_passed_count}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
	video.write(binary_diff_bgr)
	video2.write(current_frame)

video.release()
video2.release()